In [5]:
import pyodbc 
import tkinter as tk
from tkinter import *
from tkinter import ttk, filedialog
import ttkbootstrap as ttkbs
from ttkbootstrap.constants import *
import cv2
import numpy as np
from PIL import Image, ImageTk

imagem_global = None
imagem_sobel = None  # Variável global para armazenar a imagem com filtro Sobel

def abrir_imagem():
    global imagem_global, imagem_sobel  # Declara as variáveis globais
    caminho_imagem = filedialog.askopenfilename(title="Escolha uma imagem", 
                                                filetypes=[("Arquivos de imagem", "*.jpg;*.png;*.jpeg")])
    if caminho_imagem:
        img = Image.open(caminho_imagem)
        img.thumbnail((400, 400))
        img_tk = ImageTk.PhotoImage(img)
        lbl_imagem.config(image=img_tk)
        lbl_imagem.image = img_tk  # Necessário para o tkinter manter a referência
        imagem_global = np.array(img)  # Converte a imagem PIL para NumPy

        # Aplica o filtro Sobel imediatamente após carregar a imagem
        aplicar_filtro('xy')

        # Mostra os botões após carregar a imagem
        btn_frame.pack(pady=10)

def aplicar_filtro(filtro):
    global imagem_global, imagem_sobel
    if imagem_global is not None:
        imagem_gray = cv2.cvtColor(imagem_global, cv2.COLOR_RGB2GRAY)
        if filtro == 'x':
            sobel = cv2.Sobel(imagem_gray, cv2.CV_64F, 1, 0, ksize=3)
        elif filtro == 'y':
            sobel = cv2.Sobel(imagem_gray, cv2.CV_64F, 0, 1, ksize=3)
        elif filtro == 'xy':  # Aplica ambos os filtros e combina
            sobel_x = cv2.Sobel(imagem_gray, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(imagem_gray, cv2.CV_64F, 0, 1, ksize=3)
            sobel = np.sqrt(sobel_x**2 + sobel_y**2)

        sobel_normalizado = np.uint8(255 * (sobel - np.min(sobel)) / (np.max(sobel) - np.min(sobel)))
        imagem_sobel = Image.fromarray(sobel_normalizado)
        imagem_sobel.thumbnail((400, 400))
        img_sobel_tk = ImageTk.PhotoImage(imagem_sobel)
        
        lbl_imagem_sobel.config(image=img_sobel_tk)
        lbl_imagem_sobel.image = img_sobel_tk

def remover_filtro():
    global imagem_global
    if imagem_global is not None:
        img_tk = ImageTk.PhotoImage(Image.fromarray(imagem_global))
        lbl_imagem_sobel.config(image=img_tk)
        lbl_imagem_sobel.image = img_tk  # Atualiza a imagem

def aplicar_filtro_xy():
    aplicar_filtro('xy')  # Reaplica o filtro Sobel XY

# Cria a janela principal
janela = ttkbs.Window(themename="darkly")
janela.title("Tela de Seleção de Imagem")
janela.configure(bg="#333")

# Define o tamanho da janela e a centraliza
janela.geometry("900x500")
largura_tela = janela.winfo_screenwidth()
altura_tela = janela.winfo_screenheight()
pos_x = (largura_tela // 2) - (900 // 2)
pos_y = (altura_tela // 2) - (500 // 2)
janela.geometry(f"900x500+{pos_x}+{pos_y}")

menu_principal = Menu(janela)
janela.config(menu=menu_principal)

menu_opcoes = Menu(menu_principal, tearoff=0, bg="#1f4b99", fg="white", activebackground="#4a6fa5", activeforeground="white")
menu_principal.add_cascade(label="Menu", menu=menu_opcoes)
menu_opcoes.add_command(label="Add imagem", command=abrir_imagem)
menu_opcoes.add_command(label="Sair", command=janela.destroy)

# Layout principal
frame = tk.Frame(janela, bg="#333")
frame.pack(pady=20)

lbl_imagem = tk.Label(frame, bg="#333")
lbl_imagem.grid(row=0, column=0, padx=(20, 10))  # Imagem original

lbl_imagem_sobel = tk.Label(frame, bg="#333")
lbl_imagem_sobel.grid(row=0, column=1, padx=(10, 20))  # Imagem com filtro

# Botões para aplicar filtros e remover filtro (inicialmente ocultos)
btn_frame = tk.Frame(janela, bg="#333")

btn_sobel_x = tk.Button(btn_frame, text="Filtro Sobel em X", command=lambda: aplicar_filtro('x'), bg="#1f4b99", fg="white")
btn_sobel_x.pack(side=tk.LEFT, padx=5)

btn_sobel_y = tk.Button(btn_frame, text="Filtro Sobel em Y", command=lambda: aplicar_filtro('y'), bg="#1f4b99", fg="white")
btn_sobel_y.pack(side=tk.LEFT, padx=5)

btn_sobel_xy = tk.Button(btn_frame, text="Filtro Sobel XY", command=aplicar_filtro_xy, bg="#1f4b99", fg="white")
btn_sobel_xy.pack(side=tk.LEFT, padx=5)

btn_remover_filtro = tk.Button(btn_frame, text="Remover Filtro", command=remover_filtro, bg="#1f4b99", fg="white")
btn_remover_filtro.pack(side=tk.LEFT, padx=5)

# Inicia o loop da interface gráfica
janela.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog
import ttkbootstrap as ttkbs
from ttkbootstrap.constants import *
import cv2
import numpy as np
from PIL import Image, ImageTk
from tkinter import Scale

class ImageProcessor:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Image Processing Tool")
        self.setup_variables()
        self.create_gui()
        self.setup_bindings()
        
    def setup_variables(self):
        self.imagem_global = None
        self.imagem_filtrada = None
        self.zoom_history = []
        self.zoom_ativo = False
        self.last_filter = None
        self.start_x = None
        self.start_y = None
        self.rect_id = None
        
        # Bilateral filter parameters
        self.d_value = tk.IntVar(value=9)
        self.sigma_color = tk.DoubleVar(value=75.0)
        self.sigma_space = tk.DoubleVar(value=75.0)
        
        # LoG parameters
        self.gaussian_sigma = tk.DoubleVar(value=1.0)
        
        #Gaussian filter parameters
        self.kernel_size = tk.IntVar(value=111)  # Valor inicial do tamanho do kernel

    def create_gui(self):
        # Criando o canvas e a scrollbar para a janela inteira
        self.canvas = tk.Canvas(self.root)
        self.canvas.pack(side="left", fill=tk.BOTH, expand=True)

        # Adicionando a scrollbar no lado direito
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        # Criando um frame rolável dentro do canvas
        self.scrollable_frame = ttk.Frame(self.canvas)
        
        # Colocando o frame dentro do canvas
        self.canvas_window = self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")

        

        # Main container with dark theme
        self.main_frame = ttk.Frame(self.scrollable_frame, padding="10")
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Top menu
        self.create_menu()
        
        # Image display area
        self.create_canvas_area()
        
        # Control panel
        self.create_control_panel()
        
        # Parameters panel
        self.create_parameters_panel()
        
        # Atualizando a área de rolagem do canvas quando o conteúdo mudar
        self.scrollable_frame.bind("<Configure>", self.on_frame_configure)
        self.canvas.bind("<Configure>", self.configurar_canvas)
        self.canvas.bind_all("<MouseWheel>", self._on_mousewheel)

    def _on_mousewheel(self, event):
        """Permite rolar usando o mouse wheel"""
        self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")

    def configurar_canvas(self, event):
        """Ajusta o tamanho do frame interior ao canvas"""
        canvas_width = event.width
        self.canvas.itemconfig(
            self.canvas_window,
            width=canvas_width
        )

    def create_menu(self):
        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)
        
        file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Open Image", command=self.abrir_imagem)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)
        
    def create_canvas_area(self):
        self.canvas_frame = ttk.Frame(self.main_frame)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Left canvas (original image)
        self.canvas_imagem = tk.Canvas(self.canvas_frame, width=500, height=500, 
                                     bg="#2b2b2b", highlightthickness=1)
        self.canvas_imagem.grid(row=0, column=0, padx=5)
        
        # Right canvas (filtered image)
        self.canvas_sobel = tk.Canvas(self.canvas_frame, width=500, height=500, 
                                    bg="#2b2b2b", highlightthickness=1)
        self.canvas_sobel.grid(row=0, column=1, padx=5)
        
    def create_control_panel(self):
        control_frame = ttk.LabelFrame(self.main_frame, text="Controls", padding="5")
        control_frame.pack(fill=tk.X, pady=5)
        
        # Filter buttons
        filters_frame = ttk.Frame(control_frame)
        filters_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(filters_frame, text="Sobel X", 
                  command=lambda: self.aplicar_filtro('x')).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="Sobel Y", 
                  command=lambda: self.aplicar_filtro('y')).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="Sobel XY", 
                  command=lambda: self.aplicar_filtro('xy')).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="Laplacian", 
                  command=self.aplicar_laplaciano).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="LoG", 
                  command=self.aplicar_log).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="Bilateral", 
                  command=self.aplicar_bilateral).pack(side=tk.LEFT, padx=2)
        ttk.Button(filters_frame, text="Gaussiano", 
                  command=self.aplicar_gaussiano).pack(side=tk.LEFT, padx=2)
        
        # Zoom controls
        zoom_frame = ttk.Frame(control_frame)
        zoom_frame.pack(fill=tk.X, pady=5)
        
        self.zoom_btn = ttk.Button(zoom_frame, text="Enable Zoom", 
                                 command=self.toggle_zoom)
        self.zoom_btn.pack(side=tk.LEFT, padx=2)
        
        ttk.Button(zoom_frame, text="Undo Zoom", 
                  command=self.undo_zoom).pack(side=tk.LEFT, padx=2)
        ttk.Button(zoom_frame, text="Reset Image", 
                  command=self.reset_image).pack(side=tk.LEFT, padx=2)
                  
    def create_parameters_panel(self):
        param_frame = ttk.LabelFrame(self.main_frame, text="Filter Parameters", padding="5")
        param_frame.pack(fill=tk.X, pady=5)
        
        # Bilateral filter parameters
        bilateral_frame = ttk.LabelFrame(param_frame, text="Bilateral Filter", padding="5")
        bilateral_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(bilateral_frame, text="Diametro:").grid(row=0, column=0, sticky='ew')

        # Função para atualizar o Label com o valor do slider
        def update_diameter_label(*args):
            diameter_label.config(text=str(self.d_value.get()))
        def update_color_label(*args):
            diameter_label2.config(text=str(self.sigma_color.get()))
    
        def update_space_label(*args):
            diameter_label3.config(text=str(self.sigma_space.get()))

        # Slider para o "Diameter"
        Scale(bilateral_frame, from_=1, to=200, orient=tk.HORIZONTAL, 
            variable=self.d_value).grid(row=0, column=1, sticky='ew')

        # Label para mostrar o valor do slider ao lado do slider
        diameter_label = ttk.Label(bilateral_frame, text=str(self.d_value.get()))  # Inicializa com o valor atual
        diameter_label.grid(row=0, column=2, sticky='w')  # Coloca ao lado do slider
        



        # Vincula a atualização do Label ao movimento do slider
        self.d_value.trace_add("write", update_diameter_label)
        
        ttk.Label(bilateral_frame, text="Color Sigma:").grid(row=1, column=0, sticky='w')
        Scale(bilateral_frame, from_=1, to=300, orient=tk.HORIZONTAL, 
              variable=self.sigma_color).grid(row=1, column=1, sticky='ew')
        diameter_label2 = ttk.Label(bilateral_frame, text=str(self.sigma_color.get()))
        diameter_label2.grid(row=1, column=2, sticky='w')  # Coloca ao lado do slider
        
        self.sigma_color.trace_add("write", update_color_label)

        ttk.Label(bilateral_frame, text="Space Sigma:").grid(row=2, column=0, sticky='w')
        Scale(bilateral_frame, from_=1, to=300, orient=tk.HORIZONTAL, 
              variable=self.sigma_space).grid(row=2, column=1, sticky='ew')
        diameter_label3 = ttk.Label(bilateral_frame, text=str(self.sigma_space.get()))
        diameter_label3.grid(row=2, column=2, sticky='w')  # Coloca ao lado do slider

        self.sigma_space.trace_add("write", update_space_label)
        
        # LoG parameters
        log_frame = ttk.LabelFrame(param_frame, text="LoG Filter", padding="5")
        log_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(log_frame, text="Gaussian Sigma:").grid(row=0, column=0, sticky='w')
        Scale(log_frame, from_=0.1, to=5.0, orient=tk.HORIZONTAL, resolution=0.1,
              variable=self.gaussian_sigma).grid(row=0, column=1, sticky='ew')
              
    def setup_bindings(self):
        self.canvas_imagem.bind("<Button-1>", lambda e: self.iniciar_selecao(e, "original"))
        self.canvas_imagem.bind("<B1-Motion>", lambda e: self.atualizar_selecao(e, "original"))
        self.canvas_imagem.bind("<ButtonRelease-1>", lambda e: self.finalizar_selecao(e, "original"))
        
        self.canvas_sobel.bind("<Button-1>", lambda e: self.iniciar_selecao(e, "filtrada"))
        self.canvas_sobel.bind("<B1-Motion>", lambda e: self.atualizar_selecao(e, "filtrada"))
        self.canvas_sobel.bind("<ButtonRelease-1>", lambda e: self.finalizar_selecao(e, "filtrada"))

    def abrir_imagem(self):
        caminho_imagem = filedialog.askopenfilename(
            title="Choose Image",
            filetypes=[("Image files", "*.jpg;*.png;*.jpeg;*.bmp;*.tiff")]
        )
        if caminho_imagem:
            self.carregar_imagem(caminho_imagem)
            
    def carregar_imagem(self, caminho):
        img = Image.open(caminho)
        img.thumbnail((500, 500))
        self.imagem_global = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        self.atualizar_canvas_original()
        self.aplicar_filtro('xy')  # Apply default filter
        
    def atualizar_canvas_original(self):
        if self.imagem_global is not None:
            img = Image.fromarray(cv2.cvtColor(self.imagem_global, cv2.COLOR_BGR2RGB))
            img_tk = ImageTk.PhotoImage(img)
            self.canvas_imagem.delete("all")
            self.canvas_imagem.create_image(250, 250, image=img_tk, anchor='center')
            self.canvas_imagem.image = img_tk
            
    def aplicar_filtro(self, tipo):
        if self.imagem_global is None:
            return
            
        self.last_filter = tipo
        img_gray = cv2.cvtColor(self.imagem_global, cv2.COLOR_BGR2GRAY)
        
        if tipo == 'x':
            resultado = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
        elif tipo == 'y':
            resultado = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
        elif tipo == 'xy':
            sobel_x = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
            resultado = np.sqrt(sobel_x**2 + sobel_y**2)
            
        resultado = cv2.normalize(resultado, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        self.imagem_filtrada = cv2.cvtColor(resultado, cv2.COLOR_GRAY2BGR)
        self.atualizar_canvas_filtrado()
        
    def aplicar_laplaciano(self):
        if self.imagem_global is None:
            return
            
        self.last_filter = 'laplacian'
        img_gray = cv2.cvtColor(self.imagem_global, cv2.COLOR_BGR2GRAY)
        resultado = cv2.Laplacian(img_gray, cv2.CV_64F)
        resultado = cv2.normalize(resultado, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        self.imagem_filtrada = cv2.cvtColor(resultado, cv2.COLOR_GRAY2BGR)
        self.atualizar_canvas_filtrado()
        
    def aplicar_log(self):
        if self.imagem_global is None:
            return
            
        self.last_filter = 'log'
        img_gray = cv2.cvtColor(self.imagem_global, cv2.COLOR_BGR2GRAY)
        sigma = self.gaussian_sigma.get()
        
        # Apply Gaussian blur
        img_blur = cv2.GaussianBlur(img_gray, (0, 0), sigma)
        
        # Apply Laplacian
        resultado = cv2.Laplacian(img_blur, cv2.CV_64F)
        resultado = cv2.normalize(resultado, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        self.imagem_filtrada = cv2.cvtColor(resultado, cv2.COLOR_GRAY2BGR)
        self.atualizar_canvas_filtrado()
        
    def aplicar_bilateral(self):
        if self.imagem_global is None:
            return
            
        self.last_filter = 'bilateral'
        d = self.d_value.get()
        sigma_color = self.sigma_color.get()
        sigma_space = self.sigma_space.get()
        
        self.imagem_filtrada = cv2.bilateralFilter(
            self.imagem_global, d, sigma_color, sigma_space
        )
        self.atualizar_canvas_filtrado()

    def aplicar_gaussiano(self):
        if self.imagem_global is None:
            return
        
        self.last_filter = 'gaussiano'
        
        # Obter o valor do kernel do slider
        kernel_size = self.kernel_size.get()
        sigma = self.gaussian_sigma.get()
        
        # Garantir que o kernel seja ímpar e maior que 0
        if kernel_size % 2 == 0:
            kernel_size += 1
        
        print(f"Aplicando Gaussiano - Kernel: {kernel_size}, Sigma: {sigma}")  # Debug
        
        try:
            # Aplicar o filtro Gaussiano
            self.imagem_filtrada = cv2.GaussianBlur(
                src=self.imagem_global,
                ksize=(kernel_size, kernel_size),
                sigmaX=sigma,
                sigmaY=sigma
            )
            
            # Atualizar o canvas com a imagem filtrada
            self.atualizar_canvas_filtrado()
        except Exception as e:
            print(f"Erro ao aplicar filtro Gaussiano: {e}")

    def atualizar_canvas_filtrado(self):
        if self.imagem_filtrada is not None:
            img = Image.fromarray(cv2.cvtColor(self.imagem_filtrada, cv2.COLOR_BGR2RGB))
            img_tk = ImageTk.PhotoImage(img)
            self.canvas_sobel.delete("all")
            self.canvas_sobel.create_image(250, 250, image=img_tk, anchor='center')
            self.canvas_sobel.image = img_tk
            
    def toggle_zoom(self):
        self.zoom_ativo = not self.zoom_ativo
        if self.zoom_ativo:
            self.zoom_btn.configure(text="Disable Zoom")
        else:
            self.zoom_btn.configure(text="Enable Zoom")
            
    def iniciar_selecao(self, event, tipo):
        if not self.zoom_ativo:
            return
            
        self.start_x = event.x
        self.start_y = event.y
        canvas = event.widget
        self.rect_id = canvas.create_rectangle(
            self.start_x, self.start_y, self.start_x, self.start_y,
            outline="red", tags="selection"
        )
        
    def atualizar_selecao(self, event, tipo):
        if not self.zoom_ativo or self.rect_id is None:
            return
            
        canvas = event.widget
        canvas.coords(self.rect_id, self.start_x, self.start_y, event.x, event.y)
        
    def finalizar_selecao(self, event, tipo):
        if not self.zoom_ativo or self.rect_id is None:
            return
            
        canvas = event.widget
        coords = canvas.coords(self.rect_id)
        canvas.delete("selection")
        
        if len(coords) == 4:
            x1, y1, x2, y2 = map(int, coords)
            img_source = self.imagem_global if tipo == "original" else self.imagem_filtrada
            
            # Store current state for undo
            self.zoom_history.append((self.imagem_global.copy(), self.imagem_filtrada.copy()))
            
            # Get original image dimensions
            height, width = img_source.shape[:2]
            
            # Calculate display dimensions (maintaining aspect ratio)
            display_width = 500
            display_height = 500
            aspect_ratio = width / height
            
            if aspect_ratio > 1:  # Landscape image
                display_height = int(500 / aspect_ratio)
            else:  # Portrait image
                display_width = int(500 * aspect_ratio)
            
            # Calculate scale factors based on actual display size
            scale_x = width / display_width
            scale_y = height / display_height
            
            # Calculate padding for centered image
            pad_x = (500 - display_width) // 2
            pad_y = (500 - display_height) // 2
            
            # Adjust coordinates based on padding
            x1 = max(0, x1 - pad_x)
            x2 = max(0, x2 - pad_x)
            y1 = max(0, y1 - pad_y)
            y2 = max(0, y2 - pad_y)
            
            # Convert display coordinates to image coordinates
            x1_real = int(min(x1, x2) * scale_x)
            y1_real = int(min(y1, y2) * scale_y)
            x2_real = int(max(x1, x2) * scale_x)
            y2_real = int(max(y1, y2) * scale_y)
            
            # Ensure coordinates are within image bounds
            x1_real = max(0, min(x1_real, width))
            x2_real = max(0, min(x2_real, width))
            y1_real = max(0, min(y1_real, height))
            y2_real = max(0, min(y2_real, height))
            
            # Extract the region
            region = img_source[y1_real:y2_real, x1_real:x2_real]
            if region.size > 0:
                # Update the global image with the zoomed region
                self.imagem_global = cv2.resize(region, (width, height))
                
                # Reapply the last filter if necessary
                if self.last_filter:
                    if self.last_filter == 'bilateral':
                        self.aplicar_bilateral()
                    elif self.last_filter == 'laplacian':
                        self.aplicar_laplaciano()
                    elif self.last_filter == 'log':
                        self.aplicar_log()
                    else:
                        self.aplicar_filtro(self.last_filter)
                else:
                    self.imagem_filtrada = cv2.resize(region, (width, height))
                
                # Update both canvases
                self.atualizar_canvas_original()
                self.atualizar_canvas_filtrado()
        
        self.rect_id = None
        
    def undo_zoom(self):
        if self.zoom_history:
            # Restore the last state
            self.imagem_global, self.imagem_filtrada = self.zoom_history.pop()
            self.atualizar_canvas_original()
            self.atualizar_canvas_filtrado()
            
    def reset_image(self):
        if self.zoom_history:
            # Restore the initial state
            self.imagem_global, self.imagem_filtrada = self.zoom_history[0]
            self.zoom_history.clear()
            self.atualizar_canvas_original()
            self.atualizar_canvas_filtrado()
            
    def on_frame_configure(self, event):
        # Atualiza a região visível do canvas para o tamanho do conteúdo
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))

# Create and run the application
if __name__ == "__main__":
    root = ttkbs.Window(themename="darkly")
    root.title("Advanced Image Processing Tool")
    
    # Set initial window size and position
    window_width = 1200
    window_height = 800
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_position = (screen_width - window_width) // 2
    y_position = (screen_height - window_height) // 2
    

    root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")
    
    app = ImageProcessor(root)
    root.mainloop()

Aplicando Gaussiano - Kernel: 21, Sigma: 1.0
Aplicando Gaussiano - Kernel: 21, Sigma: 1.0
